# SELECT AND FILTER entities

## Setup

Import [azure data tables](https://pypi.org/project/azure-data-tables/) (version 12.2.0 Updated nov.2021)
> pip install azure-data-tables

In [1]:
from azure.data.tables import TableClient, UpdateMode
from azure.core.exceptions import ResourceExistsError, HttpResponseError, ResourceNotFoundError


INSERT ENVIRONMENT VARIABLES

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
connection = os.getenv("YOUR_CONNECTION_STRING") # AZURE_STORAGE_TABLE
table = "test"

In [4]:
# Define function for create
def entity_crud(connection_string, table_name, operation, entity):
    with TableClient.from_connection_string(connection_string, table_name) as table_client:
        if operation == 'create':
            try:
                response = table_client.create_entity(entity=entity)
                return(response)
            except ResourceExistsError:
                print("Entity already exists")

In [5]:
# Input test data

my_data = {}
my_data['PartitionKey'] = "ThisIsATest"
my_data['RowKey'] = "1"
my_data['firstName'] = "Sam"
my_data['surName'] = "Harris"
my_data['age'] = 57
my_data['woman'] = False
print(my_data)
entity_crud(connection_string=connection, table_name=table, operation='create', entity=my_data)

two_my_data = {}
two_my_data['PartitionKey'] = "ThisIsATest"
two_my_data['RowKey'] = "2"
two_my_data['firstName'] = "Camala"
two_my_data['surName'] = "Harris"
two_my_data['age'] = 59
two_my_data['woman'] = True
print(two_my_data)
entity_crud(connection_string=connection, table_name=table, operation='create', entity=two_my_data)

three_my_data = {}
three_my_data['PartitionKey'] = "ThisIsATest"
three_my_data['RowKey'] = "3"
three_my_data['firstName'] = "Eric"
three_my_data['surName'] = "Hanson"
three_my_data['age'] = 45
three_my_data['woman'] = False
print(three_my_data)
entity_crud(connection_string=connection, table_name=table, operation='create', entity=three_my_data)

{'PartitionKey': 'ThisIsATest', 'RowKey': '1', 'firstName': 'Sam', 'surName': 'Harris', 'age': 57, 'woman': False}
Entity already exists
{'PartitionKey': 'ThisIsATest', 'RowKey': '2', 'firstName': 'Camala', 'surName': 'Harris', 'age': 59, 'woman': True}
Entity already exists
{'PartitionKey': 'ThisIsATest', 'RowKey': '3', 'firstName': 'Eric', 'surName': 'Hanson', 'age': 45, 'woman': False}
Entity already exists


## Function

### Query multiple with SELECT

In [6]:
def list_entities(connection_string, table_name, select):
    with TableClient.from_connection_string(connection_string, table_name=table_name) as table_client:
        try:
            entities = list(table_client.list_entities(select=select))
            return entities
        except HttpResponseError as e:
                print(e.message)

### Query with FILTER

In [3]:
def sample_query_entities_values(connection_string,table_name, select, parameters, filter):
    with TableClient.from_connection_string(connection_string, table_name) as table_client:
        try:
            queried_entities = table_client.query_entities(
                query_filter=filter, select=[select], parameters=parameters
            )
            for entity_chosen in queried_entities:
                print(entity_chosen)
        except HttpResponseError as e:
            print(e.message)

## Playtime
<p>Now that we have our SELECT multiple, lets try it out</p>

### SELECT

In [7]:
# We can pass a single string as the select parameter
response = list_entities(connection_string=connection, table_name=table, select="PartitionKey")
print(response)

AttributeError: 'NoneType' object has no attribute 'strip'

As expected, a list with the data we selected for is returned. <br>
PS. The result is a list of dictionaries

In [9]:
# We can also pass a list of strings (column names)
response = list_entities(connection_string=connection, table_name=table, select=["firstName","surName","age"])
print(response)

[{'age': 57, 'firstName': 'Sam', 'surName': 'Harris'}, {'age': 59, 'firstName': 'Camala', 'surName': 'Harris'}, {'age': 45, 'firstName': 'Eric', 'surName': 'Hanson'}]


### SELECT & FILTER

In [5]:
# We search for all entities with an age greater than 25 and less than 50.
parameters = {u"lower": 25, u"upper": 50}
filter = u"age gt @lower and age lt @upper"
select = u"age, firstName, surName"

sample_query_entities_values(connection,table, select, parameters, filter)

AttributeError: 'NoneType' object has no attribute 'strip'

As we can se there are quite a bit of setup to be able to select and filter. <br>
We need to know the colum name, the column data type, and the [operators](https://docs.microsoft.com/en-us/rest/api/storageservices/querying-tables-and-entities#supported-comparison-operators) available. <br>
Depending on the use case, we might just filter the data in our program instead.

##### SUPPORTED QUERY OPTIONS
- $filter<br>
Returns only tables or entities that satisfy the specified filter. <br>
- $top<br>
Returns only the top n tables or entities from the set. <br>
- $select<br>
Returns the desired properties of an entity from the set. This query option is only supported for requests using version 2011-08-18 or newer.<br>

#### Supported Comparison Operators
Within a $filter clause, you can use comparison operators to specify the criteria against which to filter the query results.

For all property types, the following comparison operators are supported:


| Operator           | URI expression |
|--------------------|----------------|
| Equal              | eq             |
| GreaterThan        | gt             |
| GreaterThanOrEqual | ge             |
| LessThan           | lt             |
| LessThanOrEqual    | le             |
| NotEqual           | ne             |

Additionally, the following operators are supported for Boolean properties:

| Operator | URI expression |
|----------|----------------|
| And      | and            |
| Not      | not            |
| Or       | or             |